In [1]:
import numpy as np
import matplotlib

import matplotlib.pyplot as plt
import psoap
from psoap.data import gwori, lkca14
from psoap import matrix_functions
from psoap import covariance

from matplotlib.ticker import FormatStrFormatter as FSF
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import MultipleLocator


In [12]:
order = 22
wl0 = 5212
wl1 = 5240
# order = 49
# wl0 = 8780
# wl1 = 8800

In [13]:
a = 0.26
l = 6.0

In [14]:
lkca14.sort_by_SN()
n_epochs = 10
wl = lkca14.wl[0:n_epochs,order,:]
# Sort by ind
ind = (wl[0] > wl0) & (wl[0] < wl1)

wl = wl[:,ind]
fl = lkca14.fl[0:n_epochs,order,ind]
sigma = lkca14.sigma[0:n_epochs,order,ind]
date1D = lkca14.date1D[0:n_epochs]

In [15]:
fl_2 = np.empty_like(fl)
fl_2[:] = fl

In [16]:
fig, ax = plt.subplots()
for i in range(n_epochs):
    ax.plot(wl[i], fl[i])
plt.show()

In [17]:
# determine the calibration polynomials for each order
fl_cal = covariance.cycle_calibration(wl, fl, sigma, a, l, ncycles=3, order=2, limit_array=4, soften=1.0)

In [20]:
mu_prior, Sigma = covariance.predict_one(wl.flatten(), fl.flatten(), sigma.flatten(), wl.flatten(), a, l)
# Sigma[np.diag_indices_from(Sigma)] += 1e-10 * np.ones(N)

In [21]:
mu_post, Sigma = covariance.predict_one(wl.flatten(), fl_cal.flatten(), sigma.flatten(), wl.flatten(), a, l)

In [24]:
fig, ax = plt.subplots(nrows=5, sharex=True, figsize=(3.5, 5.5))

ax[0].axhline(1.0, ls=":", color="0.5")
ax[1].axhline(1.0, ls=":", color="0.5")
ax[2].axhline(1.0, ls=":", color="0.5")


for i in range(n_epochs):
    ax[0].plot(wl[i], fl_2[i], lw=1)

    
    ax[2].plot(wl[i], fl_cal[i]/fl[i], lw=1)
    
    ax[3].plot(wl[i], fl_cal[i], lw=1)
#     ax[4].plot() # residuals

resid = fl.flatten() - mu_prior
resid_cal = fl_cal.flatten() - mu_post

# Total squared error
print("Resid ", np.mean(resid*resid))
print("Resid cal", np.mean(resid_cal*resid_cal))

ax[1].plot(wl.flatten(), resid, ".", color="0.5") # residuals
ax[1].set_ylim(-0.4, 0.4)
ax[4].plot(wl.flatten(), resid_cal, ".", color="0.5")
ax[4].set_ylim(-0.4, 0.4)    
    
ax[-1].set_xlabel(r"$\lambda\;[\AA]$")

fig.savefig("calibration.pdf")
fig.savefig("calibration.png")

Resid  0.00417123575502
Resid cal 0.00385066450633


In [15]:
# Make a better plot of each epoch, showing original, vs new.
for i in range(n_epochs):
    fig, ax = plt.subplots(nrows=2, sharex=True)

    wl_orig = wl[i]
    fl_orig = fl[i]
    
    wl_rest = np.delete(wl, i, axis=0)
    fl_rest = np.delete(fl, i, axis=0)
    
    fl_c = fl_cal[i]
    fl_c_rest = np.delete(fl_cal, i, axis=0)
    
    for j in range(n_epochs - 1):
        ax[0].plot(wl_rest[j], fl_c_rest[j], lw=0.8, color="0.5")
        
    ax[0].plot(wl_orig, fl_orig, "k")
    ax[0].plot(wl_orig, fl_c, "m")
    
    fig.savefig("moving_spec_{}.png".format(i))
    
    plt.close("all")

/home/ian/.build/anaconda/lib/python3.5/site-packages/matplotlib/pyplot.py:516: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
